In [1]:
import os
import pandas as pd
import subprocess as sp

In [2]:
homepath = os.path.expanduser('~') + '/'

In [3]:
samtoolspath = homepath + 'tools/samtools-1.5/'
bedtoolspath = homepath + 'tools/bedtools2.26/bin/'
bam = homepath + '180502_bamQC_pedagogia/389.bam'
name = '389'
bedfile = homepath + 'nextera-dna-exome-targeted-regions-manifest-v1-2_fixed.bed'
stats_file = '%s389_samtools_stats.log' %(homepath)
final_stats = '389_final_stats.tsv'

In [ ]:
# samtools STATS
cmd = '%ssamtools stats %s > %s' %(samtoolspath, bam, stats_file)
sp.run(cmd, shell=True)

In [ ]:
# transform BAM to BED
cmd = '%sbamToBed -i %s > %s.bed' %(bedtoolspath, bam, name)
sp.run(cmd, shell=True)

In [4]:
# get OFF-TARGET alignments
cmd = '%sintersectBed -a %s.bed -b %s -v | wc -l' %(bedtoolspath, name, bedfile)
off_target = float(sp.run(cmd, shell=True, stdout=sp.PIPE).stdout.decode())

In [5]:
off_target

25093506.0

In [179]:
with open(stats_file) as infile:
    stats = infile.read().splitlines()

stats = [e.replace(':', '').split('\t') for e in stats if 'SN' in e][1:]

indeces = []
dic = {}
for e in stats:
    label = e[1].replace(' ', '_')
    dic[label] = float(e[2])
    indeces.append(label)

cmd = "cat %s | awk -F'\t' 'BEGIN{SUM=0}{ SUM+=$3-$2 }END{print SUM}'" %bedfile
dic['exome_size'] = float(sp.run(cmd, shell=True, stdout=sp.PIPE).stdout.decode().splitlines()[0])
indeces.append('exome_size')

dic['reads_off_target'] = off_target
indeces.append('reads_off_target')

ot = dic['reads_off_target']/dic['sequences']
bm = dic['bases_mapped']
mapq0 = dic['reads_MQ0']/dic['sequences']
d = dic['reads_duplicated']/dic['sequences']
es = dic['exome_size']

dic['coverage'] = bm * (1 - ot) * (1 - d) * (1 - mapq0) / es
indeces.append('coverage')

final_stats = pd.Series(dic, index=indeces)
final_stats.to_csv('389_final_stats.tsv', sep='\t', header=False)

In [183]:
dic
pd.Series(dic, index=indeces)

{'raw_total_sequences': 57419520.0,
 'filtered_sequences': 0.0,
 'sequences': 57419520.0,
 'is_sorted': 1.0,
 '1st_fragments': 28709760.0,
 'last_fragments': 28709760.0,
 'reads_mapped': 57364148.0,
 'reads_mapped_and_paired': 57333018.0,
 'reads_unmapped': 55372.0,
 'reads_properly_paired': 57194524.0,
 'reads_paired': 57419520.0,
 'reads_duplicated': 6978972.0,
 'reads_MQ0': 5676731.0,
 'reads_QC_failed': 0.0,
 'non-primary_alignments': 10879.0,
 'total_length': 4363883520.0,
 'bases_mapped': 4359675248.0,
 'bases_mapped_(cigar)': 4347172369.0,
 'bases_trimmed': 0.0,
 'bases_duplicated': 530401872.0,
 'mismatches': 15081438.0,
 'error_rate': 0.003469252,
 'average_length': 76.0,
 'maximum_length': 76.0,
 'average_quality': 34.7,
 'insert_size_average': 170.3,
 'insert_size_standard_deviation': 71.9,
 'inward_oriented_pairs': 28381244.0,
 'outward_oriented_pairs': 16081.0,
 'pairs_with_other_orientation': 3728.0,
 'pairs_on_different_chromosomes': 35514.0,
 'exome_size': 45315485.0,
 

raw_total_sequences               5.741952e+07
filtered_sequences                0.000000e+00
sequences                         5.741952e+07
is_sorted                         1.000000e+00
1st_fragments                     2.870976e+07
last_fragments                    2.870976e+07
reads_mapped                      5.736415e+07
reads_mapped_and_paired           5.733302e+07
reads_unmapped                    5.537200e+04
reads_properly_paired             5.719452e+07
reads_paired                      5.741952e+07
reads_duplicated                  6.978972e+06
reads_MQ0                         5.676731e+06
reads_QC_failed                   0.000000e+00
non-primary_alignments            1.087900e+04
total_length                      4.363884e+09
bases_mapped                      4.359675e+09
bases_mapped_(cigar)              4.347172e+09
bases_trimmed                     0.000000e+00
bases_duplicated                  5.304019e+08
mismatches                        1.508144e+07
error_rate   

In [57]:
path = homepath + '180427_bamQC_pedagogia/read_alignment.log'

off_target = pd.read_table(path, sep='\t', header=0)

In [69]:
path = homepath + '389_bam.stats'

with open(path) as infile:
    stats389 = infile.read().splitlines()

stats389 = [e.replace(':', '').split('\t') for e in stats389 if 'SN' in e][1:]
dic389 = {}

for e in stats389:
    dic389[e[1]] = float(e[2])

In [70]:
dic389
stats389

{'raw total sequences': 57419520.0,
 'filtered sequences': 0.0,
 'sequences': 57419520.0,
 'is sorted': 1.0,
 '1st fragments': 28709760.0,
 'last fragments': 28709760.0,
 'reads mapped': 57364148.0,
 'reads mapped and paired': 57333018.0,
 'reads unmapped': 55372.0,
 'reads properly paired': 57194524.0,
 'reads paired': 57419520.0,
 'reads duplicated': 6978972.0,
 'reads MQ0': 5676731.0,
 'reads QC failed': 0.0,
 'non-primary alignments': 10879.0,
 'total length': 4363883520.0,
 'bases mapped': 4359675248.0,
 'bases mapped (cigar)': 4347172369.0,
 'bases trimmed': 0.0,
 'bases duplicated': 530401872.0,
 'mismatches': 15081438.0,
 'error rate': 0.003469252,
 'average length': 76.0,
 'maximum length': 76.0,
 'average quality': 34.7,
 'insert size average': 170.3,
 'insert size standard deviation': 71.9,
 'inward oriented pairs': 28381244.0,
 'outward oriented pairs': 16081.0,
 'pairs with other orientation': 3728.0,
 'pairs on different chromosomes': 35514.0}

[['SN', 'raw total sequences', '57419520'],
 ['SN', 'filtered sequences', '0'],
 ['SN', 'sequences', '57419520'],
 ['SN', 'is sorted', '1'],
 ['SN', '1st fragments', '28709760'],
 ['SN', 'last fragments', '28709760'],
 ['SN', 'reads mapped', '57364148'],
 ['SN',
  'reads mapped and paired',
  '57333018',
  '# paired-end technology bit set + both mates mapped'],
 ['SN', 'reads unmapped', '55372'],
 ['SN', 'reads properly paired', '57194524', '# proper-pair bit set'],
 ['SN', 'reads paired', '57419520', '# paired-end technology bit set'],
 ['SN', 'reads duplicated', '6978972', '# PCR or optical duplicate bit set'],
 ['SN', 'reads MQ0', '5676731', '# mapped and MQ=0'],
 ['SN', 'reads QC failed', '0'],
 ['SN', 'non-primary alignments', '10879'],
 ['SN', 'total length', '4363883520', '# ignores clipping'],
 ['SN', 'bases mapped', '4359675248', '# ignores clipping'],
 ['SN', 'bases mapped (cigar)', '4347172369', '# more accurate'],
 ['SN', 'bases trimmed', '0'],
 ['SN', 'bases duplicated', '